In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
df = pd.read_csv("/Users/sujitharajan/Documents/GitHub/IS460/data/df1.csv")


In [2]:
from sklearn.model_selection import train_test_split
X = df.drop('price_boxcox_scaled', axis=1)
y = df['price_boxcox_scaled']
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.2)
print(X_train.shape)
print(y_test.shape)

(238021, 10)
(59506,)


In [3]:
# Reshaping input to be 3D [samples, timesteps, features] for LSTM
X_train_array = X_train.values
X_test_array = X_test.values
X_train_reshaped = X_train_array.reshape((X_train_array.shape[0], 1, X_train_array.shape[1]))
X_test_reshaped = X_test_array.reshape((X_test_array.shape[0], 1, X_test_array.shape[1]))

In [4]:
model = Sequential()

In [5]:
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
model.add(Dropout(0.2))

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [6]:
model.add(Dense(units=1))

In [7]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [8]:
model.fit(X_train_reshaped, y_train, epochs=100, batch_size=128)

Epoch 1/100
1860/1860 ━━━━━━━━━━━━━━━━━━━━ 2s 675us/step - loss: 1.0471
Epoch 2/100
1860/1860 ━━━━━━━━━━━━━━━━━━━━ 1s 656us/step - loss: 0.9983
Epoch 3/100
1860/1860 ━━━━━━━━━━━━━━━━━━━━ 1s 659us/step - loss: 1.0010
Epoch 4/100
1860/1860 ━━━━━━━━━━━━━━━━━━━━ 1s 652us/step - loss: 0.9985
Epoch 5/100
1860/1860 ━━━━━━━━━━━━━━━━━━━━ 1s 647us/step - loss: 1.0019
Epoch 6/100
1860/1860 ━━━━━━━━━━━━━━━━━━━━ 1s 664us/step - loss: 1.0013
Epoch 7/100
1860/1860 ━━━━━━━━━━━━━━━━━━━━ 1s 637us/step - loss: 1.0029
Epoch 8/100
1860/1860 ━━━━━━━━━━━━━━━━━━━━ 1s 644us/step - loss: 1.0011
Epoch 9/100
1860/1860 ━━━━━━━━━━━━━━━━━━━━ 1s 676us/step - loss: 1.0034
Epoch 10/100
1860/1860 ━━━━━━━━━━━━━━━━━━━━ 1s 699us/step - loss: 0.9977
Epoch 11/100
1860/1860 ━━━━━━━━━━━━━━━━━━━━ 1s 670us/step - loss: 1.0005
Epoch 12/100
1860/1860 ━━━━━━━━━━━━━━━━━━━━ 1s 674us/step - loss: 0.9999
Epoch 13/100
1860/1860 ━━━━━━━━━━━━━━━━━━━━ 1s 671us/step - loss: 1.0042
Epoch 14/100
1860/1860 ━━━━━━━━━━━━━━━━━━━━ 1s 652us/step - 

In [9]:
y_test_pred = model.predict(X_test_reshaped)
y_train_pred = model.predict(X_train_reshaped)
y_test_pred = y_test_pred.reshape(y_test_pred.shape[0], -1)
y_train_pred = y_train_pred.reshape(y_train_pred.shape[0], -1)

1860/1860 ━━━━━━━━━━━━━━━━━━━━ 1s 285us/step
7439/7439 ━━━━━━━━━━━━━━━━━━━━ 2s 239us/step


In [10]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error

train_mse = mean_squared_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)
test_mape = mean_absolute_percentage_error(y_test, y_test_pred)

In [11]:
lstm_results = pd.DataFrame(['LSTM', train_mse, train_r2, test_mse, test_r2,test_mape]).transpose()
lstm_results.columns = ['Method', 'Training MSE', 'Training R2', 'Test MSE', 'Test R2','Test MAPE']
lstm_results

,Method,Training MSE,Training R2,Test MSE,Test R2,Test MAPE
0,LSTM,0.999988,0.000062,0.996215,0.0001,1.006348
